In [1]:
##Preparing data as per https://doi.org/10.1186/s12920-018-0460-9 
##Predicting drug response of tumors from integrated genomic profiles by deep neural networks
##Yu-Chiao Chiu, Hung-I Harry Chen, Tinghe Zhang, Songyao Zhang, Aparna Gorthi, Li-Ju Wang, Yufei Huang

In [136]:
import pandas as pd
import numpy as np
import h5py

In [3]:
#TCGA_exp = pd.read_csv('counts_gene.tsv', delimiter='\t')  --> TCGA exp downloaded from GDC Portal is not used , it contains 58.000 genes, most of them from intron, 
#while those downloaded from https://amp.pharm.mssm.edu/archs4/download.html  contains a selection of 25.150 genes

In [4]:
#print(TCGA_exp.shape)
#TCGA_exp.columns[-1]

In [5]:
#TCGA_exp.loc[TCGA_exp['3DFF72D2-F292-497E-ACE3-6FAA9C884205'] == 439935,'gene_id']

In [164]:
import mygene
mg = mygene.MyGeneInfo()
mg.getgene('ENSGR0000223511', 'name,symbol,refseq.rna') 

In [162]:
mg.getgene(150465, 'name,symbol,refseq.rna')

{'_id': '150465',
 '_score': 14.277923,
 'name': 'tubulin tyrosine ligase',
 'refseq': {'rna': ['NM_153712.5', 'XM_005263599.3', 'XM_011510665.2']},
 'symbol': 'TTL'}

In [8]:
#Importing TCGA RNA expression data downloaded from  https://amp.pharm.mssm.edu/archs4/download.html
filename = "../tcga_matrix.h5"

h5 = h5py.File(filename,'r')
for k in h5.keys():
    print(k)


meta = h5['meta']  
dat = h5['data']  
info = h5['info']  



#Showing which type of data it contains    
for s in meta.keys():
    print('meta')
    print(s)
    print(meta[s].shape)
    print(meta[s].value)
    
for s in dat.keys():
    print('data')
    print(s)
    print(dat[s].shape)
    print(dat[s].value)    
    
for s in info.keys():
    print(s)
    print(info[s].shape)
    print(info[s].value)    

#h5.close()

meta
data
info
meta
auc
(11284,)
[b'3116748351' b'5069849036' b'4882401623' ... b'5742234142' b'6773612787'
 b'7613399024']
meta
bigwig_file
(11284,)
[b'3DFF72D2-F292-497E-ACE3-6FAA9C884205.bw'
 b'B1E54366-42B9-463C-8615-B34D52BD14DC.bw'
 b'473713F7-EB41-4F20-A37F-ACD209E3CB75.bw' ...
 b'76987B28-B56B-4A1F-B77C-1E08B8EFEF90.bw'
 b'33737781-8638-4FA2-AD4C-E888BB9343D8.bw'
 b'0BB05CA7-C6FF-42A1-919C-D801F471CBBD.bw']
meta
cancertype
(11284,)
[b'Liver Hepatocellular Carcinoma' b'Prostate Adenocarcinoma'
 b'Rectum Adenocarcinoma' ... b'Brain Lower Grade Glioma'
 b'Acute Myeloid Leukemia' b'Thyroid Carcinoma']
meta
gdc_annotations
(11284,)
[b'NULL' b'NULL' b'NULL' ... b'NULL'
 b'list(category = "Alternate sample pipeline", status = "Approved", entity_id = "fff35c80-88cd-4923-80c1-0273ba5bed0f", classification = "Notification", updated_datetime = "2016-05-01T15:00:21.638875-05:00", created_datetime = "2012-11-13T12:37:22-05:00", entity_submitter_id = "TCGA-AB-2881", notes = "Biospecimens fro

 [  40426   40053      48 ...  793314  481256  313210]]
author
(1,)
[b'Alexander Lachmann']
contact
(1,)
[b'alexander.lachmann@mssm.edu']
countsource
(1,)
[b'recount2: https://jhubiostatistics.shinyapps.io/recount/']
creationdate
(1,)
[b'10/9/2017']
lab
(1,)
[b"Ma'ayan Lab, Icahn School of Medicine"]


In [9]:
#if not exists it saves a file 'log2expression.npy' with the log2 transformation of RNA expression 
import os.path
def f(x):
    #return log2(TPM+0.001)
    return np.log2(x +0.001 ) 

l = None
if not os.path.exists('../log2expression.npy'):
    #log2(TPM+0.001) expression values computed per gene
    l = dat['expression'].value
    l = np.vectorize(f)(l)
    np.save('log2expression.npy', l)
else: 
    l = np.load('../log2expression.npy')

from scipy import stats
stats.describe(l)

DescribeResult(nobs=11284, minmax=(array([ 4.90693868,  4.39238612, -9.96578428, ..., 14.11845455,
       12.98815228, 11.60733078]), array([25.74434345, 23.45781508, 21.78148379, ..., 24.41667745,
       22.1506869 , 21.65159744])), mean=array([14.51133518, 14.65828565,  7.81866386, ..., 19.87816157,
       18.0625713 , 17.54948585]), variance=array([ 5.12112323,  3.17381026, 58.31272768, ...,  1.31941998,
        1.13436237,  0.97464454]), skewness=array([ 1.34799737,  1.15626794, -0.97460423, ..., -0.09400296,
       -0.34315363, -0.48017077]), kurtosis=array([4.18004613, 4.2873423 , 0.95231487, ..., 0.81602377, 0.62193682,
       0.94856403]))

In [10]:
#associate gene names and case_id with RNA expression 
def f(x):
    return x.decode("utf-8")
genes = np.vectorize(f)(meta['genes'].value)
cases_id  = np.vectorize(f)(meta['gdc_cases.case_id'].value)
df = pd.DataFrame(l, columns = genes, index= cases_id)


In [11]:
print(df.shape)
TCGA_exp_matrix = df.transpose()

(11284, 25150)


In [221]:
TCGA_exp_matrix.index

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A3GALT2', 'A4GALT', 'A4GNT',
       ...
       'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', length=25150)

In [266]:
TCGA_exp_matrix.head()

,0004d251-3f70-4395-b175-c94c2f5b1b81,000d566c-96c7-4f1c-b36e-fa2222467983,0011a67b-1ba9-4a32-a6b8-7850759a38cf,001887aa-36d0-463f-8bca-dec7043b4f2e,001944e5-af34-4061-9c09-bb9ea346f6fd,001ad307-4ad3-4f1d-b2fc-efc032871c7e,001cef41-ff86-4d3f-a140-a647ac4b10a1,001e0309-9c50-42b0-9e38-347883ee2cd3,0020317d-d10e-4e75-8fa6-7c1bdcdee471,0022478c-4dfd-4cbe-a05e-fb20310844e3,...,ffc915b8-cacd-4974-a040-ee496f0efc0e,ffcec8e5-9fd3-4b42-a7cb-74761f713cf4,ffcf851d-7fa1-4b45-911a-a3fbd74c253a,ffcfa005-a04f-458e-9d1d-86143dd823e5,ffd8d31f-bc4b-4e19-bbaf-0e26e9f3a107,ffed886e-261e-464c-aa27-8204dd0eb9a1,ffedc8be-1056-4205-b9d9-99b5bdb872db,fff304a2-113f-499d-a88c-9d3660c348d9,fff35c80-88cd-4923-80c1-0273ba5bed0f,fffdb1d9-58d1-425c-ac12-1e1e5f443bf7
A1BG,20.472352,14.818083,12.077484,24.457885,11.676398,14.645940,15.400379,13.695772,14.038147,14.115044,...,15.345717,15.536976,14.736032,13.498226,13.232571,12.581672,16.849882,13.367688,14.782896,15.302996
A1BG-AS1,18.746931,14.545749,13.246592,22.684448,12.256504,14.378566,15.846739,14.244587,14.291459,14.604553,...,15.947135,14.917326,14.334623,14.558959,13.628104,14.109586,15.387243,14.019591,15.667195,15.289623
A1CF,18.272466,7.584970,17.234948,19.907039,8.076821,7.087473,3.170085,-9.965784,8.103293,17.663509,...,7.348737,7.169935,-9.965784,11.264443,8.044400,18.175852,8.581204,7.781366,7.294630,5.584993
A2M,19.290201,21.170981,20.949892,23.860669,19.457004,20.946840,21.112040,20.440912,23.292556,23.420979,...,13.473072,21.439713,21.394626,21.015806,19.940004,23.392733,21.218710,21.573083,14.287207,21.750083
A2M-AS1,15.257498,17.082887,14.822969,19.778088,14.529736,16.453608,15.636710,15.083853,17.677018,18.686196,...,12.610794,15.933967,15.730231,16.734763,15.548039,17.644948,16.825431,16.116161,13.214016,16.446017


In [138]:
#CCLE_NAME only match CELL_LINE_NAME if "_" is removed from CELL_LINE_NAME and then mapped to  CCLE_ID removing _*
CCLE_annot = pd.read_csv('DepMap-2019q1-celllines_v2.csv')
CCLE_annot.columns

Index(['DepMap_ID', 'CCLE_Name', 'Aliases', 'COSMIC_ID', 'Sanger ID',
       'Primary Disease', 'Subtype Disease', 'Gender', 'Source'],
      dtype='object')

In [209]:
#GDSC (not used) -> sanger1018_brainarray_ensemblgene_rma.txt, other gen_drug ML predictor: FORESEE https://academic.oup.com/bioinformatics/advance-article/doi/10.1093/bioinformatics/btz145/5367831 
#CCLE exp -> v21.data.gex_avg_log2.txt, v21.meta.gex_features.txt is downloaded from ftp://caftpd.nci.nih.gov/pub/OCG-DCC/CTD2/Broad/CTRPv2.0_2015_ctd2_ExpandedDataset/ 
#CCLE_read_exp = pd.read_csv('CTRPv2.1_2016_pub_NatChemBiol_12_109/v21.data.gex_avg_log2.txt', delimiter='\t')
CCLE_read_exp = pd.read_csv('CCLE_depMap_19Q1_TPM.csv', index_col=0)


In [210]:
CCLE_exp = pd.merge(CCLE_annot[['CCLE_Name','DepMap_ID']],CCLE_read_exp, left_on=['DepMap_ID'], right_on=CCLE_read_exp.index )


In [211]:
CCLE_exp.drop(columns=['DepMap_ID'], inplace = True)

In [279]:
CCLE_exp.head()


,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP11-309M23.1 (ENSGR0000237531),AMDP1 (ENSGR0000237801),BX649553.1 (ENSGR0000263835),BX649553.2 (ENSGR0000263980),BX649553.3 (ENSGR0000264510),BX649553.4 (ENSGR0000264819),RN7SL355P (ENSGR0000265350),MIR3690 (ENSGR0000265658),AL732314.1 (ENSGR0000266731),AJ271736.10 (ENSGR0000270726)
NIHOVCAR3,5.851999,0.000000,7.390083,2.097611,4.253233,0.042644,0.879706,5.666188,4.542258,4.033863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HL60,0.163499,0.000000,5.659639,1.238787,3.040892,4.250962,0.163499,4.168321,4.026800,2.821710,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CACO2,6.054631,0.084064,7.760487,1.851999,3.875780,0.000000,0.056584,6.594399,4.571070,4.157852,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HEL,2.615887,0.000000,5.323370,2.405992,3.902074,0.925999,4.888987,3.896272,4.835924,4.986866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HEL9217,3.066950,0.000000,5.762615,2.992768,5.359662,0.238787,5.700162,4.143230,5.380591,5.339137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [213]:
CCLE_exp['CELL_LINE_NAME'] = CCLE_exp['CCLE_Name'].str.replace(r'_.*','')

In [214]:
CCLE_exp.index = CCLE_exp['CELL_LINE_NAME']
CCLE_exp.drop(columns=['CELL_LINE_NAME'], inplace = True)


In [215]:
CCLE_exp.drop(columns=['CCLE_Name'], inplace = True)


In [216]:
CCLE_matrix = CCLE_exp.transpose()


In [217]:
CCLE_matrix.columns.name = None


In [218]:
CCLE_matrix['t'] = CCLE_matrix.index

In [219]:
CCLE_matrix['new_index'] = CCLE_matrix['t'].str.replace(r'(\.)*\s*(\(.*)*','')


TSPAN6 (ENSG00000000003)                     TSPAN6
TNMD (ENSG00000000005)                         TNMD
DPM1 (ENSG00000000419)                         DPM1
SCYL3 (ENSG00000000457)                       SCYL3
C1orf112 (ENSG00000000460)                 C1orf112
FGR (ENSG00000000938)                           FGR
CFH (ENSG00000000971)                           CFH
FUCA2 (ENSG00000001036)                       FUCA2
GCLC (ENSG00000001084)                         GCLC
NFYA (ENSG00000001167)                         NFYA
STPG1 (ENSG00000001460)                       STPG1
NIPAL3 (ENSG00000001461)                     NIPAL3
LAS1L (ENSG00000001497)                       LAS1L
ENPP4 (ENSG00000001561)                       ENPP4
SEMA3F (ENSG00000001617)                     SEMA3F
CFTR (ENSG00000001626)                         CFTR
ANKIB1 (ENSG00000001629)                     ANKIB1
CYP51A1 (ENSG00000001630)                   CYP51A1
KRIT1 (ENSG00000001631)                       KRIT1
RAD52 (ENSG0

In [222]:
CCLE_matrix.index = CCLE_matrix['new_index']
CCLE_matrix.drop(columns=['new_index','t'], inplace=True)


,NIHOVCAR3,HL60,CACO2,HEL,HEL9217,MONOMAC6,LS513,C2BBE1,NCIH2077,253J,...,RC2,NB1643,TTC442,RH36,RH4,RH18DM,HCC827GR5,NCIH2882,JURKAT,A101D
new_index,,,,,,,,,,,,,,,,,,,,,
TSPAN6,5.851999,0.163499,6.054631,2.615887,3.066950,0.176323,4.066089,6.507953,4.462052,4.577731,...,4.718636,4.110196,5.609105,5.756223,4.562548,5.326609,6.368245,1.090853,0.014355,3.485427
TNMD,0.000000,0.000000,0.084064,0.000000,0.000000,0.000000,0.000000,0.097611,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.014355,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DPM1,7.390083,5.659639,7.760487,5.323370,5.762615,6.127221,5.889960,7.981624,7.236493,5.535742,...,6.778734,6.001577,7.605257,7.651339,7.207112,6.978310,6.337889,6.845239,5.750607,7.107060
SCYL3,2.097611,1.238787,1.851999,2.405992,2.992768,2.720278,3.049631,2.247928,2.364572,2.087463,...,2.307429,2.321928,2.272023,2.446256,2.121015,2.693766,2.097611,1.718088,2.575312,2.166715
C1orf112,4.253233,3.040892,3.875780,3.902074,5.359662,4.195348,3.760221,4.491212,4.532317,2.646163,...,3.977280,3.025029,4.076388,4.131754,4.608218,4.260026,4.146492,3.782409,5.130931,4.113534
FGR,0.042644,4.250962,0.000000,0.925999,0.238787,3.884598,0.028569,0.000000,0.028569,0.000000,...,0.000000,0.028569,0.389567,0.000000,0.028569,0.000000,0.042644,0.000000,0.622930,0.000000
CFH,0.879706,0.163499,0.056584,4.888987,5.700162,3.834913,0.014355,0.028569,3.279471,0.356144,...,6.201242,0.097611,6.219362,2.618239,4.395748,0.389567,0.056584,0.042644,0.056584,0.097611
FUCA2,5.666188,4.168321,6.594399,3.896272,4.143230,5.256633,5.561937,7.088523,6.718636,5.525129,...,4.969473,2.632268,6.401733,3.910733,5.599615,3.490570,5.734168,6.034304,0.263034,6.433460
GCLC,4.542258,4.026800,4.571070,4.835924,5.380591,4.906410,4.974070,5.727376,4.225738,4.354029,...,3.206331,4.043519,3.924100,3.080658,2.292782,3.285402,4.047015,5.511911,4.811985,4.847496


In [223]:
del CCLE_matrix.index.name 

In [246]:
#the same genes need to be on both CCLE_exp and TCGA_exp
exp_genes = list(set(TCGA_exp_matrix.index) & set(CCLE_matrix.index))
CCLE_matrix_filtered = CCLE_matrix.loc[exp_genes, :]
CCLE_matrix_f = CCLE_matrix_filtered[~CCLE_matrix_filtered.index.duplicated(keep='first')]
TCGA_exp_matrix_filtered = TCGA_exp_matrix.loc[exp_genes, :]
TCGA_exp_matrix_f = TCGA_exp_matrix_filtered[~TCGA_exp_matrix_filtered.index.duplicated(keep='first')]
print(CCLE_matrix_filtered.shape)
print(TCGA_exp_matrix_filtered.shape)
print(CCLE_matrix_f.shape)
print(TCGA_exp_matrix_f.shape)

(22858, 1165)
(22374, 11284)
(22374, 1165)
(22374, 11284)


In [260]:
CCLE_matrix_f.loc['A1BG','22RV1']

1.72246602447

In [272]:
#IC50 for CCLE as provided by authors after imputation
IC50_CCLE_imputed = pd.read_csv('ic50_data_imputed.txt', delimiter='\t', index_col=0, header=0)

In [273]:
del IC50_CCLE_imputed.index.name

In [27]:
#IC50 for CCLE obtained from supplement material in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4967469/#mmc5 
#Caveat of this source: even if it has a larger number of cells (990), many of them are NaN, so imputation of values for 
#those cell lines and use them as ground truth seems too inacurate. 
IC50 = pd.read_excel('mmc5.xlsx', sheet_name='TableS4A-IC50s',  skiprows = range(1, 5), header=1, index_col=1)


In [28]:

IC50.drop(columns= 'Unnamed: 0', inplace = True)
IC50.index = IC50.index.str.replace('-','')


In [280]:
IC50 = IC50.transpose()
IC50.head()

,TL-2-105,TAK-715,CP466722,BMS-345541,Genentech Cpd 10,GSK429286A,Ruxolitinib,SB-715992,ZSTK474,KIN001-102,...,Salubrinal,Dasatinib,XMD8-85,Roscovitine,Lapatinib,Cyclopamine,VX-680,JW-7-52-1,Erlotinib,Rapamycin
ALLPO,1.412465,2.113150,-0.756692,1.386112,-1.398149,3.681923,3.081088,-3.230122,-1.691070,-0.722249,...,1.395198,1.371619,1.499415,3.886316,2.623544,2.219550,-1.518877,-2.901511,1.314559,-1.747501
AMO1,3.363134,3.051047,1.718606,2.191936,1.541583,3.544126,2.989808,-3.888980,0.275043,1.626300,...,3.281077,1.245299,2.330333,3.517712,2.402243,4.386527,0.828576,-1.850099,2.758619,-2.294581
COLO668,3.398414,3.103017,3.067645,1.228204,0.313308,3.572447,3.731025,-2.275653,0.577291,0.940925,...,3.941262,2.764619,2.630081,3.610561,0.992627,5.317472,3.252960,-1.380020,1.787143,-2.310652
CORL95,3.312450,6.096484,2.950619,2.848541,3.740211,5.216102,3.719220,-0.931930,2.301070,2.685076,...,4.383999,3.406481,3.749075,5.317720,3.303382,5.049294,3.417887,-0.338626,3.865993,-0.099652
DG75,3.650059,3.107754,2.175176,1.587807,2.438582,5.301156,4.153156,-2.527416,2.322550,2.099412,...,3.031548,0.832321,1.822337,3.786396,3.315752,4.746855,2.372698,-1.485584,3.316530,-1.462775


In [30]:
#cell lines without IC50 for specific drugs are inputed with kNN (k = 5) using KNN from VIM package of R (other jupyter to be uploaded)


In [31]:
print(IC50.shape) #drug response data of 990 CCLE cell lines to 265 anti-cancer
#drugs measured by the half maximal inhibitory concentration (IC50) from the GDSC Project.

(265, 990)


In [32]:
#MAF - Fields:entity_id, case_id, Hugo_Symbol
#four types of nonsynonymous mutations, including missense and nonsense mutations, frameshift insertions and deletions


In [105]:
#CCLE MAF uses the newest version in 2019 from https://depmap.org/portal/download/all/?release=DepMap+Public+19Q1&file=DepMap-2019q1-celllines_v2.csv
#CCLE MAF downloaded from https://data.broadinstitute.org/ccle/CCLE_DepMap_18q3_maf_20180718.txt is not longer used
CCLE_maf_read = pd.read_csv('depmap_19Q1_mutation_calls.csv')

/home/auri/anaconda3/envs/gen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,20,23,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
CCLE_maf = pd.merge(CCLE_annot[['CCLE_Name','DepMap_ID']],CCLE_maf_read, left_on=['DepMap_ID'], right_on=['DepMap_ID'] )

In [109]:
CCLE_maf.drop(columns=['DepMap_ID'], inplace = True)

In [110]:
CCLE_maf['CELL_LINE_NAME'] = CCLE_maf['CCLE_Name'].str.replace(r'_.*','')

In [111]:
#select only 4 types of mutations: four types of nonsynonymous mutations, including missense and nonsense mutations, 
#frameshift insertions and deletions --> only those are tagged as mutated genes in the binary matrices
print(CCLE_maf.columns)
set(CCLE_maf['Variant_Classification'].values)
print(CCLE_maf.groupby(['Variant_Classification','Variant_Type']).count())

Index(['CCLE_Name', 'Unnamed: 0', 'Hugo_Symbol', 'Entrez_Gene_Id',
       'NCBI_Build', 'Chromosome', 'Start_position', 'End_position', 'Strand',
       'Variant_Classification', 'Variant_Type', 'Reference_Allele',
       'Tumor_Seq_Allele1', 'dbSNP_RS', 'dbSNP_Val_Status', 'Genome_Change',
       'Annotation_Transcript', 'Tumor_Sample_Barcode', 'cDNA_Change',
       'Codon_Change', 'Protein_Change', 'isDeleterious', 'isTCGAhotspot',
       'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt', 'ExAC_AF', 'VA_WES_AC',
       'CGA_WES_AC', 'SangerWES_AC', 'SangerRecalibWES_AC', 'RNAseq_AC',
       'HC_AC', 'RD_AC', 'WGS_AC', 'Variant_annotation', 'CELL_LINE_NAME'],
      dtype='object')
                                       CCLE_Name  Unnamed: 0  Hugo_Symbol  \
Variant_Classification   Variant_Type                                       
3'UTR                    DEL                  49          49           49   
                         INS                  12          12           12   
5'Fla

In [112]:
CCLE_maf = CCLE_maf[['Variant_Classification', 'Hugo_Symbol', 'CELL_LINE_NAME']]

In [113]:
mutated = ['Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins', 'Missense_Mutation','Nonsense_Mutation']
CCLE_maf.loc[:,'Mutated']   = 0
CCLE_maf.loc[CCLE_maf['Variant_Classification'].isin(mutated),'Mutated'] = 1

In [274]:
CCLE_maf.Mutated.value_counts()



1    796738
0    446407
Name: Mutated, dtype: int64

In [115]:
CCLE_maf.drop(columns=['Variant_Classification'], inplace = True)

In [116]:
CCLE_maf_matrix = CCLE_maf.groupby([ 'CELL_LINE_NAME','Hugo_Symbol']).count()

In [117]:
CCLE_maf_matrix = CCLE_maf_matrix.unstack(level=0)

In [118]:
CCLE_maf_matrix.shape

(19350, 1585)

In [119]:
del CCLE_maf_matrix.index.name
CCLE_maf_matrix.columns = CCLE_maf_matrix.columns.droplevel() 
CCLE_maf_matrix.columns.name = None

In [120]:
CCLE_maf_matrix = CCLE_maf_matrix.fillna(value = 0) #all combinations of gene and cell line not reported in the MAF file is assumed to be wildtype 
CCLE_maf_matrix.head()

,127399,201T,22RV1,2313287,253J,253JBV,42MGBA,451LU,5637,59M,...,YD15,YD38,YD8,YH13,YKG1,YMB1E,YT,ZR751,ZR7530,[MERGED
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
A2ML1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
A3GALT2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
import os
import glob

#TCGA MAF data is obtained from GDC Portal using a manifest to download this query MAF files + open: 
#https://portal.gdc.cancer.gov/repository?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.access%22%2C%22value%22%3A%5B%22open%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_format%22%2C%22value%22%3A%5B%22MAF%22%5D%7D%7D%5D%7D
#long process 
if not os.path.exists('maf_TCGA.csv'):
    paths = glob.glob('maf_TCGA/**/*.somatic.maf.gz' , recursive=True)

    df = pd.DataFrame()
    i = 0
    for maf in paths:
        i+=1
        print(i)
        d = pd.read_csv(maf, compression='gzip', delimiter='\t', skiprows= 5)
        if d.columns[0] == 'Hugo_Symbol' :
            df = df.append(d)
        else:
            print(d.columns[0])
    df.to_csv('maf_TCGA.csv')
            
else:    
    TCGA_maf_read = pd.read_csv('maf_TCGA.csv', usecols=['Variant_Classification', 'Variant_Type','Hugo_Symbol', 'case_id'])


    

In [53]:
#select only 4 types of mutations: four types of nonsynonymous mutations, including missense and nonsense mutations, 
#frameshift insertions and deletions --> only those are tagged as mutated genes in the binary matrices
print(TCGA_maf_read.columns)
set(TCGA_maf_read['Variant_Classification'].values)
print(TCGA_maf_read.groupby(['Variant_Classification','Variant_Type']).count())

Index(['Hugo_Symbol', 'Variant_Classification', 'Variant_Type', 'case_id'], dtype='object')
                                     Hugo_Symbol  case_id
Variant_Classification Variant_Type                      
3'Flank                DEL                  6202     6202
                       INS                  1779     1779
                       SNP                 78791    78791
3'UTR                  DEL                 73092    73092
                       INS                 21134    21134
                       SNP                737635   737635
5'Flank                DEL                  2183     2183
                       INS                   732      732
                       SNP                 62431    62431
5'UTR                  DEL                  8361     8361
                       INS                  2933     2933
                       SNP                205110   205110
Frame_Shift_Del        DEL                166957   166957
Frame_Shift_Ins        INS            

In [54]:

mutated = ['Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins', 'Missense_Mutation','Nonsense_Mutation']
TCGA_maf_read.loc[:,'Mutated']   = 0
TCGA_maf_read.loc[TCGA_maf_read['Variant_Classification'].isin(mutated),'Mutated'] = 1

In [55]:
TCGA_maf_read.Mutated.value_counts()

1    6525124
0    4391042
Name: Mutated, dtype: int64

In [56]:
TCGA_maf_read.drop(columns=['Variant_Classification','Variant_Type'], inplace = True)

In [57]:
TCGA_maf_matrix = TCGA_maf_read.groupby([ 'case_id','Hugo_Symbol']).count()

In [58]:
TCGA_maf_matrix = TCGA_maf_matrix.unstack(level=0)

In [59]:
del TCGA_maf_matrix.index.name
TCGA_maf_matrix.columns = TCGA_maf_matrix.columns.droplevel() 
TCGA_maf_matrix.columns.name = None


,0004d251-3f70-4395-b175-c94c2f5b1b81,000d566c-96c7-4f1c-b36e-fa2222467983,0011a67b-1ba9-4a32-a6b8-7850759a38cf,001887aa-36d0-463f-8bca-dec7043b4f2e,001944e5-af34-4061-9c09-bb9ea346f6fd,001ad307-4ad3-4f1d-b2fc-efc032871c7e,001cef41-ff86-4d3f-a140-a647ac4b10a1,001e0309-9c50-42b0-9e38-347883ee2cd3,0022478c-4dfd-4cbe-a05e-fb20310844e3,0024ab57-4036-4b0f-b7a1-040f97787022,...,ffc73551-55e9-4bbb-bd15-76088551964b,ffc915b8-cacd-4974-a040-ee496f0efc0e,ffcec8e5-9fd3-4b42-a7cb-74761f713cf4,ffcf851d-7fa1-4b45-911a-a3fbd74c253a,ffcfa005-a04f-458e-9d1d-86143dd823e5,ffd8d31f-bc4b-4e19-bbaf-0e26e9f3a107,ffedc8be-1056-4205-b9d9-99b5bdb872db,fff304a2-113f-499d-a88c-9d3660c348d9,fff35c80-88cd-4923-80c1-0273ba5bed0f,fffdb1d9-58d1-425c-ac12-1e1e5f443bf7
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2MP1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A3GALT2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4GALT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4GNT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
TCGA_maf_matrix = TCGA_maf_matrix.fillna(value = 0) #all combinations of gene and cell line not reported in the MAF file are assumed to be wildtype 
TCGA_maf_matrix.head()

,0004d251-3f70-4395-b175-c94c2f5b1b81,000d566c-96c7-4f1c-b36e-fa2222467983,0011a67b-1ba9-4a32-a6b8-7850759a38cf,001887aa-36d0-463f-8bca-dec7043b4f2e,001944e5-af34-4061-9c09-bb9ea346f6fd,001ad307-4ad3-4f1d-b2fc-efc032871c7e,001cef41-ff86-4d3f-a140-a647ac4b10a1,001e0309-9c50-42b0-9e38-347883ee2cd3,0022478c-4dfd-4cbe-a05e-fb20310844e3,0024ab57-4036-4b0f-b7a1-040f97787022,...,ffc73551-55e9-4bbb-bd15-76088551964b,ffc915b8-cacd-4974-a040-ee496f0efc0e,ffcec8e5-9fd3-4b42-a7cb-74761f713cf4,ffcf851d-7fa1-4b45-911a-a3fbd74c253a,ffcfa005-a04f-458e-9d1d-86143dd823e5,ffd8d31f-bc4b-4e19-bbaf-0e26e9f3a107,ffedc8be-1056-4205-b9d9-99b5bdb872db,fff304a2-113f-499d-a88c-9d3660c348d9,fff35c80-88cd-4923-80c1-0273ba5bed0f,fffdb1d9-58d1-425c-ac12-1e1e5f443bf7
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2MP1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#Genes with no mutations in CCLE and TCGA samples were eliminated.
genes_mut_TCGA_CCLE = set(TCGA_maf_matrix.index.values)& set(CCLE_maf_matrix.index.values)
print(len(genes_mut_TCGA_CCLE))

18474


In [66]:
TCGA_maf_matrix_r = TCGA_maf_matrix.loc[list(genes_mut_TCGA_CCLE),:]
TCGA_maf_matrix_r.shape

(18474, 10189)

In [67]:
CCLE_maf_matrix_r = CCLE_maf_matrix.loc[list(genes_mut_TCGA_CCLE),:]
CCLE_maf_matrix_r.shape

(18474, 1414)

In [275]:
#Summary of data
print(len(set(CCLE_maf['CELL_LINE_NAME'].values)))
print(len(IC50_CCLE_imputed.columns)) #cell lines
print(len(set(CCLE_matrix.columns)))
print(len(set(CCLE_maf['Hugo_Symbol'].values)))
print("N Cell lines with MAF + expression + IC50 information")
#Counts of cell lines with MAF + expression + IC50 information
print(len( set(IC50_CCLE_imputed.columns) & set(CCLE_maf['CELL_LINE_NAME'].values) & set(CCLE_matrix.columns.values)))
#Couts of genes included in expression and MAF files
print("N genes with MAF and expression in cell lines")
print(len(set(CCLE_maf['Hugo_Symbol'].values) & set(CCLE_matrix.index.values)))
print("N genes with MAF and expression in cell lines and TCGA")
print(len(set(TCGA_exp_matrix.index) & set(TCGA_maf_matrix.index) & set(CCLE_matrix.index) & set(CCLE_maf_matrix.index)  )) 
print('N of drugs tested on the cell lines')
print(len(IC50_CCLE_imputed.index))
#Counts of tumors with TCGA MAF and gene expression
print("N tumors with TCGA MAF and gene expression")
tumors_TCGA = set(TCGA_maf_matrix.columns.values) & set(TCGA_exp_matrix.columns.values)
print(len(tumors_TCGA))


1585
622
1164
19350
N Cell lines with MAF + expression + IC50 information
610
N genes with MAF and expression in cell lines
18189
N genes with MAF and expression in cell lines and TCGA
17849
N of drugs tested on the cell lines
265
N tumors with TCGA MAF and gene expression
9595


In [69]:
TCGA_exp_matrix.head() 
TCGA_exp_matrix.to_csv('E_TCGA.csv')

In [261]:
CCLE_matrix_f.head() 
CCLE_matrix_f.to_csv('E_CCLE.csv')

In [269]:
TCGA_maf_matrix.head()
TCGA_maf_matrix.to_csv('M_TCGA.csv')

In [270]:
CCLE_maf_matrix.head()
CCLE_maf_matrix.to_csv('M_CCLE.csv')

In [271]:
IC50.head()
IC50.to_csv('IC50_CCLE.csv')

In [265]:
IC50_CCLE_imputed.to_csv('IC50_CCLE_author.csv')

In [278]:
print(len( set(IC50.columns) & set(IC50_CCLE_imputed.columns)))

567
